In [11]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
from torchvision import transforms
from PIL import Image
import os
import os.path as osp
import glob


#入力画像の処理を行う
#訓練時と推論時で処理が異なる

class ImageTransform():
    """
    画像の前処理クラス。
    画像のサイズをリサイズ
    resize: int
        リサイズ先の画像の大きさ
    mean : (RGB)
    std:(RGB)
    """

    def __init__(self, resize):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.Resize(resize), #リサイズ
                 transforms.CenterCrop(28),
                # transforms.RandomRotation(degrees=20), #ランダムに回転
                transforms.ToTensor(), #テンソルに変換
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize), #リサイズ
                transforms.CenterCrop(28),
                transforms.ToTensor(), #テンソルに変換
            ])
        }

    def __call__(self, img, phase='train'):
        """
        pahes:'train' or 'val'
        """
        return self.data_transform[phase](img)


class MyDataset(data.Dataset):
    '''
    file_list : リスト
        画像パス
    transform: object
        前処理クラスのインスタンス
    phase : 学習化テストか設定する
    '''
    def __init__(self, file_list, transform=None, phase='train'):
        self.transform = transform#前処理クラスのリスト
        self.file_list = file_list#ファイルパスのリスト
        self.phase = phase#train or val の指定

    def __len__(self):
        #画像の枚数を返す
        return len(self.file_list)
    
    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルの取得
        '''
        #index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        t_data = Image.new("RGB", img.size, (255, 255, 255))
        t_data.paste(img, mask=img.split()[3])
        if t_data.mode == "RGB":
            t_data = t_data.convert("L")

        #画像の前処理実施
        img_transformed = self.transform(
            t_data, self.phase
        )

        #画像のラベルをファイル名から抜き出す
        if self.phase == "train":
            label = int(img_path[19])
        elif self.phase == "val":
            label = int(img_path[len(os.getcwd()) + 24])
        label = torch.tensor(label, dtype=torch.int64)
        return img_transformed, label

class testDataset(data.Dataset):
    '''
    file_list : リスト
        画像パス
    transform: object
        前処理クラスのインスタンス
    phase : 学習化テストか設定する
    '''
    def __init__(self,transform=None, phase='train'):

        self.transform = transform#前処理クラスのリスト
        self.phase = phase#train or val の指定

        rootpath = "./訓練用画像/"
        target_path = osp.join(rootpath+'tests/*.png')
        
        path_list = []

        for path in glob.glob(target_path):
            path_list.append(path)

        self.file_list = path_list

        self.label_dict = {}

        with open('訓練用画像/tests.txt') as f:
            for line in f:
                line = line.replace("\n", "")
                name, label = line.split("/")
                self.label_dict[name] = int(label)
    

    def __len__(self):
        #画像の枚数を返す
        return len(self.file_list)
    
    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルの取得
        '''
        #index番目の画像をロード
        img_path = self.file_list[index]
        file_name = img_path[14:]
        label = self.label_dict[file_name]

        img = Image.open(img_path)
        t_data = Image.new("RGB", img.size, (255, 255, 255))
        t_data.paste(img, mask=img.split()[3])
        if t_data.mode == "RGB":
            t_data = t_data.convert("L")
        #画像の前処理実施
        img_transformed = self.transform(
            t_data, self.phase
        )

        label = torch.tensor(label, dtype=torch.int64)
        return img_transformed, label



class make_datapath_list():

    def _make_datapath_list(ph):
        """
        データパスを格納したリストを作成する
        """

        rootpath = os.getcwd() + "/src/訓練用画像/" + ph + "/"
        target_path = osp.join(rootpath+'**/*.png')
    
        path_list = []

        for path in glob.glob(target_path):
            path_list.append(path)

        return path_list
    

In [12]:
from __future__ import print_function

import argparse
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import QMNIST
#from tqdm import tqdm_notebook as tqdm



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.cnn1 = nn.Sequential(
            nn.Conv2d(1,32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Conv2d(32,32, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            nn.Dropout2d(p=0.20)
            )

        self.cnn2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2), stride=2),
            nn.Dropout2d(0.25)
        )

        self.cnn3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3)),
            nn.ReLU(),
            nn.Dropout2d(p=0.25),
        )

        self.fc = nn.Sequential(
            #全結合層
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.Dropout(),
            nn.Linear(128, 10)
        )
#         self.fc = nn.Sequential(
#             #全結合層
#             nn.Linear(15488, 128),
#             nn.BatchNorm1d(128),
#             nn.Dropout(p=0.2),
#             nn.Linear(128, 10)
#         )

    def forward(self, x):  

        out = self.cnn1(x)
        out = self.cnn2(out)
        out = self.cnn3(out)

        out = out.view(out.size()[0], -1)
        
        out = self.fc(out)
        out = F.softmax(out)
        return out


In [13]:
def train_model(net, dataloders_dict, criterion, optimizer, num_epochs):
    
    train_loss_list = []
    train_acc_list = []
    val_loss_list = []
    val_acc_list = []

    #初期設定
    #GPUが使えるか確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス", device)
    #モデルをGPUへ
    net.to(device)
    #ネットワークがある程度固定であれば高速化させる
    torch.backends.cudnn.benchmark = True
    #epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-----------------------------------')

        #epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() #モデルを訓練モードに
            else:
                net.eval() #モデルを検証モードに
            epoch_loss = 0.0 #epochの損失0
            epoch_corrects = 0 #epochの正解数
            #データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm.tqdm(dataloders_dict[phase]):
                #optimizerを初期化
                optimizer.zero_grad()
                inputs = inputs.to(device)
                labels = labels.to(device)
                #順伝搬(forward)計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)#損失を計算
                    _, preds = torch.max(outputs, 1) #ラベルを予測
                    #訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    #イテレーション結果の計算
                    #lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)
                    #正解の合計数を更新
                    epoch_corrects += torch.sum(preds == labels.data)
            #epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloders_dict[phase].dataset)
            print('{} Loss:{:.4f} Acc: {:.4f}'.format(phase, epoch_loss,epoch_acc))
            if phase == 'train':
                train_acc_list.append(epoch_acc)
                train_loss_list.append(epoch_loss)
            else:
                val_acc_list.append(epoch_acc)
                val_loss_list.append(epoch_loss)
    return val_loss_list,train_loss_list, val_acc_list, train_acc_list

In [ ]:
import random
seed = 42
random.seed(seed)  
np.random.seed(seed)  
# PyTorch のRNGを初期化  
torch.manual_seed(seed)

import tensorflow as tf
import torch.utils.data as data
from torch.utils.data import DataLoader


transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.RandomCrop(28),
     transforms.RandomRotation(degrees=20),
     transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, ), (0.5, ))
     ])


trainset = QMNIST(root='./',
                 train=True,
                 download=True,
                 transform=transform)
testset = QMNIST(root='./',
                train=False,
                download=True,
                transform=transform)

trainloader = DataLoader(trainset,
                         batch_size=100,
                         shuffle=True,
                         num_workers=2)
testloader = DataLoader(testset,
                        batch_size=100,
                        shuffle=False,
                        num_workers=2)

train_list = make_datapath_list._make_datapath_list("tranings")

test_dataset = MyDataset(file_list = train_list, transform=ImageTransform(32),phase='val')
test_dataloder = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle=False)


dataloders_dict = {"train": trainloader, "val": test_dataloder}

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.006)

val_loss, train_loss, val_acc, train_acc = train_model(model, dataloders_dict, criterion, optimizer, num_epochs=10)
torch.save(model.state_dict(), "mnist_cnn.pth")

  0%|                                                                                          | 0/600 [00:00<?, ?it/s]

使用デバイス cuda:0
Epoch 1/10
-----------------------------------


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
 19%|███████████████▌                                                                   | 6/32 [00:00<00:00, 55.05it/s]

train Loss:1.7840 Acc: 0.6803


  0%|                                                                                          | 0/600 [00:00<?, ?it/s]

val Loss:2.3472 Acc: 0.1110
Epoch 2/10
-----------------------------------


  8%|██████▉                                                                          | 51/600 [00:01<01:03,  8.58it/s]

In [ ]:
import os

print("os.path.dirname(__file__)                 : %r" % (os.path.dirname(__file__)))
print("os.path.abspath(__file__)                 : %r" % (os.path.abspath(__file__)))
print("os.path.dirname(os.path.abspath(__file__)): %r" % (os.path.dirname(os.path.abspath(__file__))))

In [28]:
import os

print('getcwd:      ', os.getcwd())


getcwd:       C:\Users\瀧川秀明\Workspase\ssi


NameError: name '__file__' is not defined